## test gpt, rag

In [ ]:
from dotenv import load_dotenv
import os


#### path
print("\n\n path ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ\n")
load_dotenv()
ROOT_DIR = os.getenv("ROOT_DIR")


book_DB_dir = os.path.join(ROOT_DIR, "book_DB")


print("pwd :", os.getcwd())
print("book_DB_dir :", book_DB_dir)



 path ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

pwd : z:\maeng_dir\sparta_camp\final_sparta\ipython
book_DB_dir : Z:\maeng_dir\sparta_camp\final_sparta\book_DB


## LangChain

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


#### input
user_question = "가지를 키울 토양의 화학성 수치 정보좀 줘"
user_soil = {'지번코드(PNU_Code)': '5115034022100750001',
 '시료채취년도(Any_Year)': '2024',
 '토양검정일(Exam_Day)': '20240325',
 '경지구분코드(Exam_Type)': '1',
 '대상지 지번주소(PNU_Nm)': '강원특별자치도 강릉시 강동면 모전리 75-1',
 '산도(ACID)': '6.800',
 '유효인산(VLDPHA)': '74.700',
 '유효규산(VLDSIA)': '77.000',
 '유기물(OM)': '13.000',
 '마그네슘(POSIFERT_MG)': '1.100',
 '칼륨(POSIFERT_K)': '0.174',
 '칼슘(POSIFERT_CA)': '3.500',
 '전기전도도(SELC)': '0.850'}

#### RAG
embeddings = OpenAIEmbeddings()
book_DB = Chroma(
            persist_directory = book_DB_dir,
            embedding_function = embeddings,
        )


retriever = book_DB.as_retriever()
retrieved_docs = retriever.invoke(user_question)
context = "\n".join([doc.page_content for doc in retrieved_docs])


#### LLM & prompt
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Question: {user_question}
prompt_RAG = ChatPromptTemplate.from_template(
    """
        
        user_soil: {user_soil}
        Context: {context}
        Answer:
            
        1. Context와 사용자의 토양 정보(user_soil)를 활용해서 사용자의 토양에 적합한 작물을 추천해줘
        
        
    """)


#### Chain
chain = prompt_RAG | llm
res = chain.invoke({"context": context, "user_soil": user_soil})
result_RAG = res.content

with open("../result_RAG.txt", "w") as f:
    f.write(result_RAG)

# print(result_RAG)

In [ ]:
# chain_C = prompt_C | llm
# res_C = chain_C.invoke({"result_RAG": result_RAG, "result_LLM": result_LLM})
# result_03 = res_C.content
# print("\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
# print("prompt_C")
# print(result_03)